#### Request data from King County data portal through API

In [1]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sodapy import Socrata

In [2]:
token = "xFcdi5UWfbhkilWL4Q5ByGnuF"

# Note 'None' in place of application token, and no username or password:
client = Socrata("data.kingcounty.gov", token)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("uydm-m3ym", limit=33000)

# Convert to pandas DataFrame
phyto_df = pd.DataFrame.from_records(results)

In [3]:
phyto_df.head()

,locator,site_name,sample_number,collect_date,depth_m,taxon,rank,kingdom,phylum,class,...,biovolume_mm_3_l,abundance_particles_ml,quality,sample_id,grab_id,profile_id,datasource,replicates_of,replicates,steward_note
0,LSEP01,South Plant Outfall,L75014-5 DUP,2020-07-21T00:00:00.000,1,Rhizosolenia setigera,Species,Chromista,Ochrophyta,Bacillariophyceae,...,0.4744,31.34,1,193176,192432,80506,KCEL,NaN,NaN,NaN
1,LSKQ06,Alki Outfall,L75156-6 DUP,2020-08-04T00:00:00.000,1.1,unidentified zooplankton,NaN,NaN,NaN,NaN,...,0.8434,0.5458,1,193174,193165,80663,KCEL,NaN,NaN,NaN
2,KSSK02,West Point Outfall,L75014-3,2020-07-20T00:00:00.000,1.1,unidentified 10-25 micron,NaN,NaN,NaN,NaN,...,0.2043,103.3,1,193133,192551,80496,KCEL,NaN,NaN,NaN
3,MSJN02,Vashon Outfall,L75156-8,2020-08-04T00:00:00.000,1,Protoceratium reticulatum,Species,Chromista,Myzozoa,Dinophyceae,...,1.96,52.68,1,193167,193167,80655,KCEL,NaN,NaN,NaN
4,LSKQ06,Alki Outfall,L75380-6,2020-09-09T00:00:00.000,0.98,unidentified 10-25 micron,NaN,NaN,NaN,NaN,...,0.0716,52.92,1,193171,193171,80661,KCEL,NaN,NaN,NaN


In [4]:
phyto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   locator                 33000 non-null  object
 1   site_name               33000 non-null  object
 2   sample_number           33000 non-null  object
 3   collect_date            33000 non-null  object
 4   depth_m                 32912 non-null  object
 5   taxon                   33000 non-null  object
 6   rank                    28878 non-null  object
 7   kingdom                 28878 non-null  object
 8   phylum                  28660 non-null  object
 9   class                   25043 non-null  object
 10  order                   19641 non-null  object
 11  family                  18205 non-null  object
 12  genus                   18205 non-null  object
 13  species                 8090 non-null   object
 14  biovolume_mm_3_l        33000 non-null  object
 15  ab

In [5]:
print (min(phyto_df['collect_date']))
print (max(phyto_df['collect_date']))


2015-01-20T00:00:00.000
2022-02-08T00:00:00.000


In [6]:
len(phyto_df['locator'].unique())

15

**Dropping samples that are sampled at locations without water column data**

This is something I checked manually on the King County data website.

In [7]:
# List of sampling sites with water column data and we want to keep

loc_drop = ['PENNCOVEWEST','PENNCOVEPNN001', 'PENNCOVECW','NSAJ02', 'PSUSANBUOY','PENNCOVEENT']
loc_keep = ['LSEP01', 'LSKQ06', 'KSSK02', 'MSJN02','JSUR01','KSBP01', 'LTED04', 'NSEX01','LSNT01']

#Drop the columns and do a sniff test

mask = phyto_df['locator'].isin(loc_keep)
phyto_df = phyto_df[mask]
phyto_df['locator'].unique()

array(['LSEP01', 'LSKQ06', 'KSSK02', 'MSJN02', 'JSUR01', 'KSBP01',
       'LTED04', 'NSEX01', 'LSNT01'], dtype=object)

In [8]:
phyto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29149 entries, 0 to 32999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   locator                 29149 non-null  object
 1   site_name               29149 non-null  object
 2   sample_number           29149 non-null  object
 3   collect_date            29149 non-null  object
 4   depth_m                 29061 non-null  object
 5   taxon                   29149 non-null  object
 6   rank                    25501 non-null  object
 7   kingdom                 25501 non-null  object
 8   phylum                  25304 non-null  object
 9   class                   22157 non-null  object
 10  order                   17360 non-null  object
 11  family                  16096 non-null  object
 12  genus                   16096 non-null  object
 13  species                 7160 non-null   object
 14  biovolume_mm_3_l        29149 non-null  object
 15  ab

Let's deal with dates so we can easily access it later for further data munging

In [9]:
phyto_df['collect_date'] = pd.to_datetime(phyto_df['collect_date'], yearfirst=True)
phyto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29149 entries, 0 to 32999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   locator                 29149 non-null  object        
 1   site_name               29149 non-null  object        
 2   sample_number           29149 non-null  object        
 3   collect_date            29149 non-null  datetime64[ns]
 4   depth_m                 29061 non-null  object        
 5   taxon                   29149 non-null  object        
 6   rank                    25501 non-null  object        
 7   kingdom                 25501 non-null  object        
 8   phylum                  25304 non-null  object        
 9   class                   22157 non-null  object        
 10  order                   17360 non-null  object        
 11  family                  16096 non-null  object        
 12  genus                   16096 non-null  object

The next step is to process the phytoplankton dataset into something that will be useful for us in building our predictive model. Remember, we're trying to predict whether or not Harmful Algal bloom will occur, and if yes, how intense?

Let's first calculate the biomass with each phytoplankton sample. This will be a very useful metric to determine "intensity" of HAB.

We are going to use the equation y = 127.67x^0.4496, where y is the biomass, and x is the biovolume in our phytoplankton data. The resulting biomass will be in micgC/L. 

This equation was attained from the [documentation](https://your.kingcounty.gov/dnrp/library/water-and-land/science/SalishSea-2018/17-2018-POSTER-Hannach-Using-FlowCAM-data.pdf) of the phytoplankton data on King County's website.


In [10]:
phyto_df.biovolume_mm_3_l = phyto_df.biovolume_mm_3_l.astype(float)
phyto_df['abundance_particles_ml'] = phyto_df['abundance_particles_ml'].astype(float)

phyto_df['biomass'] = 127.67*(phyto_df['biovolume_mm_3_l']**0.4496)

phyto_df.head()

,locator,site_name,sample_number,collect_date,depth_m,taxon,rank,kingdom,phylum,class,...,abundance_particles_ml,quality,sample_id,grab_id,profile_id,datasource,replicates_of,replicates,steward_note,biomass
0,LSEP01,South Plant Outfall,L75014-5 DUP,2020-07-21,1,Rhizosolenia setigera,Species,Chromista,Ochrophyta,Bacillariophyceae,...,31.3400,1,193176,192432,80506,KCEL,NaN,NaN,NaN,91.302676
1,LSKQ06,Alki Outfall,L75156-6 DUP,2020-08-04,1.1,unidentified zooplankton,NaN,NaN,NaN,NaN,...,0.5458,1,193174,193165,80663,KCEL,NaN,NaN,NaN,118.258826
2,KSSK02,West Point Outfall,L75014-3,2020-07-20,1.1,unidentified 10-25 micron,NaN,NaN,NaN,NaN,...,103.3000,1,193133,192551,80496,KCEL,NaN,NaN,NaN,62.515184
3,MSJN02,Vashon Outfall,L75156-8,2020-08-04,1,Protoceratium reticulatum,Species,Chromista,Myzozoa,Dinophyceae,...,52.6800,1,193167,193167,80655,KCEL,NaN,NaN,NaN,172.777501
4,LSKQ06,Alki Outfall,L75380-6,2020-09-09,0.98,unidentified 10-25 micron,NaN,NaN,NaN,NaN,...,52.9200,1,193171,193171,80661,KCEL,NaN,NaN,NaN,39.017349


Next, we will filter our phytoplankton data to just contain phytoplanktons that are considered Harmful Algaes. This information is attained [here](https://www.psp.wa.gov/PSmarinewatersoverview.php)

In [11]:
#Make a list of toxic phytoplanktons
HAB_list = ['Rhizosolenia','Thalassiosira', 'Chaetoceros','Akashiwo','Skeletonema','Pseudo-nitzschia',
            'Protoceratium','Ceratium','Prorocentrum','Coscinodiscus','Alexandrium','Heterosigma','Cerataulina','Thalassionema',
           'Proboscia','Phaeocystis','Actinoptychus','Eucampia','Detonula','Paralia','Kofodinium','Dinophysis',
           'Dictyocha','Dactyliosolen','Corethron','Cochlodinium','Asterionellopsis','Amylax','unidentified dinoflagellate']

HAB_list_join = '|'.join(HAB_list)

#Truncate dataframe to only contain rows with phytoplankton species that are considered Harmful Algal Blooms.

HAB_df = phyto_df[phyto_df['taxon'].str.contains(HAB_list_join,regex=True)]
HAB_df['taxon'].unique()

array(['Rhizosolenia setigera', 'Protoceratium reticulatum',
       'unidentified dinoflagellate <25 micron', 'Coscinodiscus',
       'Chaetoceros', 'Pseudo-nitzschia', 'Neoceratium (prev. Ceratium)',
       'Prorocentrum', 'Lauderia/Detonula', 'Eucampia zodiacus',
       'Dinophysis', 'Thalassiosira', 'Thalassionema nitzschioides',
       'Skeletonema costatum', 'Asterionellopsis glacialis',
       'Paralia sulcata', 'unidentified dinoflagellate >25 micron',
       'Akashiwo sanguinea', 'Actinoptychus senarius',
       'Heterosigma akashiwo', 'Dictyocha', 'Alexandrium',
       'Cerataulina pelagica', 'Amylax triacantha', 'Phaeocystis',
       'Cochlodinium'], dtype=object)

In [12]:
len(HAB_df['collect_date'].unique())

307

#### Let's load up water column data

In [13]:
import os

#get all csv filenames in our data directory

filedir = 'C:\\Users\\hanis\\flatiron\\phase3\\project\\Harmful-Algal-Bloom\\data\\water_col\\'
entries = os.listdir(filedir)


water_col_df = None

#Read the csvs and combine them into a big dataframe.
for loc in loc_keep:
    temp_df=None
    filenames = [i for i in entries if loc in i]
    temp_df = pd.concat((pd.read_csv(filedir+f,encoding='utf_16',header=1, index_col=None) for f in filenames), ignore_index=True)
    if water_col_df is None:
        water_col_df = temp_df
    else:
        water_col_df.append(temp_df, ignore_index=True)


<ipython-input-13-0398aafe6538>:15: DtypeWarning: Columns (9,25) have mixed types.Specify dtype option on import or set low_memory=False.
  temp_df = pd.concat((pd.read_csv(filedir+f,encoding='utf_16',header=1, index_col=None) for f in filenames), ignore_index=True)
<ipython-input-13-0398aafe6538>:15: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  temp_df = pd.concat((pd.read_csv(filedir+f,encoding='utf_16',header=1, index_col=None) for f in filenames), ignore_index=True)
<ipython-input-13-0398aafe6538>:15: DtypeWarning: Columns (7,9,25) have mixed types.Specify dtype option on import or set low_memory=False.
  temp_df = pd.concat((pd.read_csv(filedir+f,encoding='utf_16',header=1, index_col=None) for f in filenames), ignore_index=True)
<ipython-input-13-0398aafe6538>:15: DtypeWarning: Columns (5,11,13,15,17,19,25) have mixed types.Specify dtype option on import or set low_memory=False.
  temp_df = pd.concat((pd.read_csv(filedir+f,e

In [14]:
water_col_df.head()

,Locator,Sample_Date,Sample_Depth,UpDown,Sample_Temperature_field,ST_Qual,PAR_field,PA_Qual,PAR_Surface,PAS_Qual,...,Chla_field,CH_Qual,Salinity_field,SA_Qual,Turbidity_field,TB_Qual,light_transmission,LT_Qual,NO23_field,NO23_Qual
0,LSEP01,1/21/2015 8:24:54 AM,0.5,Down,8.1703,NaN,185.820,NaN,59.981,NaN,...,0.5842,NaN,22.1317,NaN,NaN,NaN,45.2209,NaN,NaN,NaN
1,LSEP01,1/21/2015 8:25:22 AM,1.0,Down,8.7123,NaN,45.548,NaN,63.030,NaN,...,2.3260,NaN,23.7796,NaN,NaN,NaN,74.7801,NaN,NaN,NaN
2,LSEP01,1/21/2015 8:28:09 AM,1.5,Down,9.0463,NaN,52.799,NaN,63.234,NaN,...,2.2405,NaN,26.7706,NaN,NaN,NaN,79.9493,NaN,NaN,NaN
3,LSEP01,1/21/2015 8:28:37 AM,2.0,Down,9.6219,NaN,43.576,NaN,64.197,NaN,...,2.1255,NaN,27.9431,NaN,NaN,NaN,84.6464,NaN,NaN,NaN
4,LSEP01,1/21/2015 8:28:45 AM,2.5,Down,9.7526,NaN,33.438,NaN,64.197,NaN,...,2.2459,NaN,28.2914,NaN,NaN,NaN,87.4143,NaN,NaN,NaN


In [15]:
water_col_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116958 entries, 0 to 116957
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Locator                    116958 non-null  object 
 1   Sample_Date                116958 non-null  object 
 2   Sample_Depth               116958 non-null  float64
 3   UpDown                     116958 non-null  object 
 4   Sample_Temperature_field   116958 non-null  float64
 5   ST_Qual                    0 non-null       float64
 6   PAR_field                  116958 non-null  float64
 7   PA_Qual                    0 non-null       float64
 8   PAR_Surface                115547 non-null  float64
 9   PAS_Qual                   2258 non-null    object 
 10  Density_field              116958 non-null  float64
 11  DN_Qual                    0 non-null       float64
 12  Sigma_Theta_Density_field  116958 non-null  float64
 13  SD_Qual                    0 

We don't really care about the data quality here so let's drop those columns as well as Turbidity_field column since there's not data on turbidity.

In [16]:
col_to_drop = ['ST_Qual', 'PA_Qual','PAS_Qual', 'DN_Qual','SD_Qual', 'DO_Qual',
        'CH_Qual','SA_Qual','TB_Qual', 'LT_Qual', 'NO23_Qual', 'Turbidity_field']
water_col_df.drop(col_to_drop, axis=1,inplace=True)
water_col_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116958 entries, 0 to 116957
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Locator                    116958 non-null  object 
 1   Sample_Date                116958 non-null  object 
 2   Sample_Depth               116958 non-null  float64
 3   UpDown                     116958 non-null  object 
 4   Sample_Temperature_field   116958 non-null  float64
 5   PAR_field                  116958 non-null  float64
 6   PAR_Surface                115547 non-null  float64
 7   Density_field              116958 non-null  float64
 8   Sigma_Theta_Density_field  116958 non-null  float64
 9   DO_field                   116958 non-null  float64
 10  Chla_field                 116958 non-null  float64
 11  Salinity_field             116958 non-null  float64
 12  light_transmission         116958 non-null  float64
 13  NO23_field                 75

Now, we have to decide whether we want to keep the downcast data or upcast data.

In [17]:
water_col_df.groupby('UpDown').describe()

Sample_Depth                                                       \
              count       mean        std  min   25%   50%    75%    max   
UpDown                                                                     
Down        58538.0  94.676902  54.227481  0.5  47.5  94.5  141.5  195.5   
Up          58420.0  94.247612  54.124685  0.5  47.5  94.0  141.0  195.0   

       Sample_Temperature_field             ... light_transmission           \
                          count       mean  ...                75%      max   
UpDown                                      ...                               
Down                    58538.0  10.544449  ...          90.520875  97.6689   
Up                      58420.0  10.539833  ...          90.460300  96.8672   

       NO23_field                                                               
            count      mean       std     min     25%      50%     75%     max  
UpDown                                                                          
Down      37930.0  0.340759  0.065806  0.0204  0.2941  0.34400  0.3918  0.4826  
Up        37816.0  0.340104  0.062061  0.0273  0.2961  0.34305  0.3885  0.4772  

[2 rows x 88 columns]

As you can see, there's only slight differences to the 0.1 degree between the downcast and upcast data so we can pick either. Since it seems like the downcast data has more samples, let's go with downcast.

In [18]:
mask = water_col_df['UpDown'] == 'Down'

water_col_df = water_col_df[mask]

water_col_df.head()

,Locator,Sample_Date,Sample_Depth,UpDown,Sample_Temperature_field,PAR_field,PAR_Surface,Density_field,Sigma_Theta_Density_field,DO_field,Chla_field,Salinity_field,light_transmission,NO23_field
0,LSEP01,1/21/2015 8:24:54 AM,0.5,Down,8.1703,185.820,59.981,1017.170,17.1676,11.0450,0.5842,22.1317,45.2209,NaN
1,LSEP01,1/21/2015 8:25:22 AM,1.0,Down,8.7123,45.548,63.030,1018.391,18.3858,7.3522,2.3260,23.7796,74.7801,NaN
2,LSEP01,1/21/2015 8:28:09 AM,1.5,Down,9.0463,52.799,63.234,1020.682,20.6750,8.2007,2.2405,26.7706,79.9493,NaN
3,LSEP01,1/21/2015 8:28:37 AM,2.0,Down,9.6219,43.576,64.197,1021.515,21.5058,7.5257,2.1255,27.9431,84.6464,NaN
4,LSEP01,1/21/2015 8:28:45 AM,2.5,Down,9.7526,33.438,64.197,1021.769,21.7578,7.4451,2.2459,28.2914,87.4143,NaN


In [19]:
water_col_df['Sample_Date'] = pd.to_datetime(water_col_df['Sample_Date'],dayfirst=True)
water_col_df.head()

,Locator,Sample_Date,Sample_Depth,UpDown,Sample_Temperature_field,PAR_field,PAR_Surface,Density_field,Sigma_Theta_Density_field,DO_field,Chla_field,Salinity_field,light_transmission,NO23_field
0,LSEP01,2015-01-21 08:24:54,0.5,Down,8.1703,185.820,59.981,1017.170,17.1676,11.0450,0.5842,22.1317,45.2209,NaN
1,LSEP01,2015-01-21 08:25:22,1.0,Down,8.7123,45.548,63.030,1018.391,18.3858,7.3522,2.3260,23.7796,74.7801,NaN
2,LSEP01,2015-01-21 08:28:09,1.5,Down,9.0463,52.799,63.234,1020.682,20.6750,8.2007,2.2405,26.7706,79.9493,NaN
3,LSEP01,2015-01-21 08:28:37,2.0,Down,9.6219,43.576,64.197,1021.515,21.5058,7.5257,2.1255,27.9431,84.6464,NaN
4,LSEP01,2015-01-21 08:28:45,2.5,Down,9.7526,33.438,64.197,1021.769,21.7578,7.4451,2.2459,28.2914,87.4143,NaN


In [20]:
water_col_df['Sample_time'] = [x.time() for x in water_col_df['Sample_Date']]
water_col_df['Sample_Date'] = [x.date() for x in water_col_df['Sample_Date']]

In [21]:
len(water_col_df['Sample_Date'].unique())

156

In [22]:
water_col_df['Chla_field']

0         0.5842
1         2.3260
2         2.2405
3         2.1255
4         2.2459
           ...  
116571    0.3317
116572    0.3331
116573    0.3416
116574    0.3404
116575    0.3406
Name: Chla_field, Length: 58538, dtype: float64

**Importing HABSOS Florida data**

In [23]:
habsos_df = pd.read_csv("./data/habsos_20220225.csv")
habsos_df.head()

C:\Users\hanis\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,STATE_ID,DESCRIPTION,LATITUDE,LONGITUDE,SAMPLE_DATE,SAMPLE_DEPTH,GENUS,SPECIES,CATEGORY,CELLCOUNT,...,WATER_TEMP,WATER_TEMP_UNIT,WATER_TEMP_QA,WIND_DIR,WIND_DIR_UNIT,WIND_DIR_QA,WIND_SPEED,WIND_SPEED_UNIT,WIND_SPEED_QA,OBJECTID
0,AL,Orange Beach Waterfront,30.29480,-87.57510,11-JAN-22 05.23.00.000000000 PM,NaN,Karenia,brevis,not observed,0,...,12.1,deg. C,1.0,NaN,NaN,9,NaN,NaN,9,939786
1,AL,Bear Point,30.30880,-87.52680,11-JAN-22 05.07.00.000000000 PM,NaN,Karenia,brevis,not observed,0,...,13.9,deg. C,1.0,NaN,NaN,9,NaN,NaN,9,939785
2,AL,Alabama Point,30.27694,-87.54167,10-JAN-22 05.43.00.000000000 PM,NaN,Karenia,brevis,not observed,0,...,16.6,deg. C,1.0,NaN,NaN,9,NaN,NaN,9,939778
3,AL,Florida Point A,30.26620,-87.55010,10-JAN-22 05.24.00.000000000 PM,NaN,Karenia,brevis,not observed,0,...,14.9,deg. C,1.0,NaN,NaN,9,NaN,NaN,9,939775
4,AL,Cotton Bayou,30.26940,-87.58200,10-JAN-22 05.07.00.000000000 PM,NaN,Karenia,brevis,not observed,0,...,16.1,deg. C,1.0,NaN,NaN,9,NaN,NaN,9,939780


In [24]:
habsos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190339 entries, 0 to 190338
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   STATE_ID         190339 non-null  object 
 1   DESCRIPTION      190192 non-null  object 
 2   LATITUDE         190339 non-null  float64
 3   LONGITUDE        190339 non-null  float64
 4   SAMPLE_DATE      190339 non-null  object 
 5   SAMPLE_DEPTH     186234 non-null  float64
 6   GENUS            190339 non-null  object 
 7   SPECIES          190339 non-null  object 
 8   CATEGORY         189483 non-null  object 
 9   CELLCOUNT        190339 non-null  int64  
 10  CELLCOUNT_UNIT   190339 non-null  object 
 11  CELLCOUNT_QA     190339 non-null  int64  
 12  SALINITY         97291 non-null   float64
 13  SALINITY_UNIT    97291 non-null   object 
 14  SALINITY_QA      190339 non-null  int64  
 15  WATER_TEMP       94361 non-null   float64
 16  WATER_TEMP_UNIT  94364 non-null   obje

Let's get rid of QA columns. Those don't give us important information for our purposes.

In [25]:
keep1 = [x for x in habsos_df.columns if 'QA' not in x]
keep2 = [x for x in keep1 if 'UNIT' not in x]
habsos_df = habsos_df[keep2]

habsos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190339 entries, 0 to 190338
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   STATE_ID      190339 non-null  object 
 1   DESCRIPTION   190192 non-null  object 
 2   LATITUDE      190339 non-null  float64
 3   LONGITUDE     190339 non-null  float64
 4   SAMPLE_DATE   190339 non-null  object 
 5   SAMPLE_DEPTH  186234 non-null  float64
 6   GENUS         190339 non-null  object 
 7   SPECIES       190339 non-null  object 
 8   CATEGORY      189483 non-null  object 
 9   CELLCOUNT     190339 non-null  int64  
 10  SALINITY      97291 non-null   float64
 11  WATER_TEMP    94361 non-null   float64
 12  WIND_DIR      0 non-null       float64
 13  WIND_SPEED    316 non-null     float64
 14  OBJECTID      190339 non-null  int64  
dtypes: float64(7), int64(2), object(6)
memory usage: 21.8+ MB


Let's deal with the date and time.

In [26]:
habsos_df['SAMPLE_DATE']

0         11-JAN-22 05.23.00.000000000 PM
1         11-JAN-22 05.07.00.000000000 PM
2         10-JAN-22 05.43.00.000000000 PM
3         10-JAN-22 05.24.00.000000000 PM
4         10-JAN-22 05.07.00.000000000 PM
                       ...               
190334    25-SEP-05 06.20.00.000000000 PM
190335    21-SEP-05 03.57.00.000000000 AM
190336    21-SEP-05 03.35.00.000000000 AM
190337    20-SEP-05 11.33.00.000000000 PM
190338    15-SEP-05 10.00.00.000000000 AM
Name: SAMPLE_DATE, Length: 190339, dtype: object

In [27]:
habsos_df['SAMPLE_DATE'] = [x[0:9] for x in habsos_df['SAMPLE_DATE']]

habsos_df['SAMPLE_DATE']

0         11-JAN-22
1         11-JAN-22
2         10-JAN-22
3         10-JAN-22
4         10-JAN-22
            ...    
190334    25-SEP-05
190335    21-SEP-05
190336    21-SEP-05
190337    20-SEP-05
190338    15-SEP-05
Name: SAMPLE_DATE, Length: 190339, dtype: object

In [28]:
habsos_df['SAMPLE_DATE'] = pd.to_datetime(habsos_df['SAMPLE_DATE'],format="%d-%b-%y")

print(habsos_df['SAMPLE_DATE'].min())
print(habsos_df['SAMPLE_DATE'].max())

1969-01-30 00:00:00
2068-12-23 00:00:00


In [29]:
habsos_df['SAMPLE_DATE'].dt.year.unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005, 2003, 2002, 2001, 2000,
       1999, 1997, 1996, 2004, 1998, 1995, 1994, 1993, 1992, 1991, 1990,
       1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979,
       1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 2068,
       2067, 2066, 2065, 2064, 2063, 2062, 2061, 2060, 2059, 2058, 2057,
       2056, 2055, 2054, 2053], dtype=int64)

Datetime changed 1968 and earlier year as 2068 and so on. We're not interested in those data anyway. Let's drop data earlier than 2000 and see how many we're left with.

In [30]:
habsos_df['year']= habsos_df['SAMPLE_DATE'].dt.year

mask = (habsos_df['year']<2053) & (habsos_df['year']>1999)

habsos_df = habsos_df[mask]

habsos_df['SAMPLE_DATE'].dt.year.unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005, 2003, 2002, 2001, 2000,
       2004], dtype=int64)

In [58]:
len(habsos_df[habsos_df['SAMPLE_DATE'].dt.year>2018])

22072

In [31]:
habsos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143155 entries, 0 to 190338
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   STATE_ID      143155 non-null  object        
 1   DESCRIPTION   143008 non-null  object        
 2   LATITUDE      143155 non-null  float64       
 3   LONGITUDE     143155 non-null  float64       
 4   SAMPLE_DATE   143155 non-null  datetime64[ns]
 5   SAMPLE_DEPTH  139114 non-null  float64       
 6   GENUS         143155 non-null  object        
 7   SPECIES       143155 non-null  object        
 8   CATEGORY      142363 non-null  object        
 9   CELLCOUNT     143155 non-null  int64         
 10  SALINITY      66598 non-null   float64       
 11  WATER_TEMP    64426 non-null   float64       
 12  WIND_DIR      0 non-null       float64       
 13  WIND_SPEED    316 non-null     float64       
 14  OBJECTID      143155 non-null  int64         
 15  year          143

We still have a really decent amount of data. These data are from so many location. Let's learn about locations these data are coming from

In [32]:
habsos_df['CELLCOUNT']

0               0
1               0
2               0
3               0
4               0
           ...   
190334     370000
190335      42000
190336      17000
190337      51000
190338    1619000
Name: CELLCOUNT, Length: 143155, dtype: int64

In [33]:
habsos_df['CELLCOUNT'].value_counts(normalize=True)

0          0.776955
333        0.016723
1000       0.014746
667        0.007474
2000       0.007041
             ...   
1513000    0.000007
661000     0.000007
128500     0.000007
200107     0.000007
447333     0.000007
Name: CELLCOUNT, Length: 5450, dtype: float64

In [34]:
habsos_df['CELLCOUNT'].describe()

count    1.431550e+05
mean     1.220410e+05
std      1.878148e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.894560e+08
Name: CELLCOUNT, dtype: float64

In [35]:
habsos_df['DESCRIPTION'].unique()

array(['Orange Beach Waterfront', 'Bear Point', 'Alabama Point', ...,
       'Boca Chica Beach 3/10 mi N of the Rio Grande mouth',
       'N side W end Causeway Base Port Isabel',
       'Near the south side of the Brazos Santiago Pass jetty at Boca Chica.'],
      dtype=object)

In [36]:
habsos_df.head()

,STATE_ID,DESCRIPTION,LATITUDE,LONGITUDE,SAMPLE_DATE,SAMPLE_DEPTH,GENUS,SPECIES,CATEGORY,CELLCOUNT,SALINITY,WATER_TEMP,WIND_DIR,WIND_SPEED,OBJECTID,year
0,AL,Orange Beach Waterfront,30.29480,-87.57510,2022-01-11,NaN,Karenia,brevis,not observed,0,18.8,12.1,NaN,NaN,939786,2022
1,AL,Bear Point,30.30880,-87.52680,2022-01-11,NaN,Karenia,brevis,not observed,0,18.5,13.9,NaN,NaN,939785,2022
2,AL,Alabama Point,30.27694,-87.54167,2022-01-10,NaN,Karenia,brevis,not observed,0,33.3,16.6,NaN,NaN,939778,2022
3,AL,Florida Point A,30.26620,-87.55010,2022-01-10,NaN,Karenia,brevis,not observed,0,19.4,14.9,NaN,NaN,939775,2022
4,AL,Cotton Bayou,30.26940,-87.58200,2022-01-10,NaN,Karenia,brevis,not observed,0,32.2,16.1,NaN,NaN,939780,2022


In [37]:
habsos_df['GENUS'].value_counts()

Karenia    143155
Name: GENUS, dtype: int64

### Try to get Argo floats

In [55]:
# Load libraries
import numpy as np
import xarray as xr
from argopy import DataFetcher as ArgoDataFetcher
import argopy
# import cartopy.crs as ccrs 
import matplotlib.pyplot as plt
# import cartopy.feature as cfeature
import gsw

The following function works to pull argo data using argopy. The function below is attained [here](https://github.com/oceanhackweek/ohw20-proj-pyxpcm/blob/master/pyxpcm-GoMx-example.ipynb).

In [56]:
argopy.dashboard()

In [ ]:
# float_id = [4901598,4903233,4903251,4903254,4903237,4903232,4903256]

In [72]:
float_id = [4902351,6902855,4901598,4902915,4903240,4902917,6902851,4902352,4902916,4903248,4903259,6902852,6902849,6902853\
           ,4903624,4903355,4903622,4903249,4903356,4903232,6902854,4903237,4903250,4902350,4903354,4903625,4903254,4903353,\
           4903277,4903276,4903251,4903256,4903233]

In [74]:
df = ArgoDataFetcher().float(4903624).to_dataframe()
df

,CONFIG_MISSION_NUMBER,CYCLE_NUMBER,DATA_MODE,DIRECTION,PLATFORM_NUMBER,POSITION_QC,PRES,PRES_QC,PSAL,PSAL_QC,TEMP,TEMP_QC,TIME_QC,LATITUDE,LONGITUDE,TIME
N_POINTS,,,,,,,,,,,,,,,,
0,1,1,A,A,4903624,1,4.160000,1,36.240002,1,28.655001,1,1,27.368,-89.795,2021-09-26 08:46:23
1,1,1,A,A,4903624,1,6.060000,1,36.240002,1,28.658001,1,1,27.368,-89.795,2021-09-26 08:46:23
2,1,1,A,A,4903624,1,8.160000,1,36.240002,1,28.660999,1,1,27.368,-89.795,2021-09-26 08:46:23
3,1,1,A,A,4903624,1,10.060000,1,36.240002,1,28.664000,1,1,27.368,-89.795,2021-09-26 08:46:23
4,1,1,A,A,4903624,1,12.060000,1,36.240002,1,28.665001,1,1,27.368,-89.795,2021-09-26 08:46:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9987,20,20,A,A,4903624,1,1600.040039,1,34.967899,1,4.314400,1,1,25.023,-91.641,2022-04-10 02:51:23
9988,20,20,A,A,4903624,1,1700.040039,1,34.968800,1,4.302000,1,1,25.023,-91.641,2022-04-10 02:51:23
9989,20,20,A,A,4903624,1,1800.099976,1,34.970100,1,4.291600,1,1,25.023,-91.641,2022-04-10 02:51:23


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 9991
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CONFIG_MISSION_NUMBER  9992 non-null   int32         
 1   CYCLE_NUMBER           9992 non-null   int32         
 2   DATA_MODE              9992 non-null   object        
 3   DIRECTION              9992 non-null   object        
 4   PLATFORM_NUMBER        9992 non-null   int32         
 5   POSITION_QC            9992 non-null   int32         
 6   PRES                   9992 non-null   float64       
 7   PRES_QC                9992 non-null   int32         
 8   PSAL                   9992 non-null   float64       
 9   PSAL_QC                9992 non-null   int32         
 10  TEMP                   9992 non-null   float64       
 11  TEMP_QC                9992 non-null   int32         
 12  TIME_QC                9992 non-null   int32         
 13  LAT

Try get the spreadsheet

In [70]:
# stations_df = pd.read_csv('./data/dashboard_15-04-2022_10-49-22.csv',delimiter=';',encoding='latin-1')
# stations_df.head()

In [71]:
# stations_df.info()

In [ ]:
def get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f):
    argopy.set_options(mode='standard')
    ds_points = ArgoDataFetcher(mode='standard',src='erddap').region([llon,rlon, llat,ulat, depthmin, depthmax,time_in,time_f]).to_xarray()
    ds_profiles = ds_points.argo.point2profile()
    return ds_profiles

In [52]:
# Function input parameters
# Bounding box for argos in gulf of mexico
llon=-98;rlon=-80
ulat=31;llat=18
depthmin=0;depthmax=200
# Time range
time_in='2018-01'
time_f='2020-01'
# Run forest run (this can take a while..)
ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f)

In [53]:
print(ds)

<xarray.Dataset>
Dimensions:                (N_LEVELS: 199, N_PROF: 2821)
Coordinates:
  * N_PROF                 (N_PROF) int32 173 249 766 634 ... 1276 11 2065 2105
  * N_LEVELS               (N_LEVELS) int32 0 1 2 3 4 5 ... 194 195 196 197 198
    LATITUDE               (N_PROF) float64 24.82 26.43 21.48 ... 26.4 26.12
    LONGITUDE              (N_PROF) float64 -89.41 -94.18 ... -85.51 -85.75
    TIME                   (N_PROF) datetime64[ns] 2018-01-01T01:24:26 ... 20...
Data variables:
    CONFIG_MISSION_NUMBER  (N_PROF) int32 6 5 4 1 5 8 221 4 ... 5 2 7 3 5 3 3 3
    CYCLE_NUMBER           (N_PROF) int32 217 165 48 293 140 ... 187 231 21 20
    DATA_MODE              (N_PROF) <U1 'D' 'D' 'D' 'A' 'D' ... 'D' 'D' 'D' 'D'
    DIRECTION              (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER        (N_PROF) int32 4901479 4901480 ... 4903252 4903254
    POSITION_QC            (N_PROF) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    PRES                  

In [64]:
df = ds.to_dataframe()


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 561379 entries, (0, 173) to (198, 2105)
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   CONFIG_MISSION_NUMBER  561379 non-null  int32         
 1   CYCLE_NUMBER           561379 non-null  int32         
 2   DATA_MODE              561379 non-null  object        
 3   DIRECTION              561379 non-null  object        
 4   PLATFORM_NUMBER        561379 non-null  int32         
 5   POSITION_QC            561379 non-null  int32         
 6   PRES                   336911 non-null  float32       
 7   PRES_QC                561379 non-null  int32         
 8   PSAL                   336911 non-null  float32       
 9   PSAL_QC                561379 non-null  int32         
 10  TEMP                   336911 non-null  float32       
 11  TEMP_QC                561379 non-null  int32         
 12  TIME_QC                561379 no